In [ ]:
import numpy as np
import argparse
import cv2 as cv
import subprocess
import time
import os
from yolo_utils import infer_image, show_image

In [ ]:
FLAGS = []

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('-w', '--weights',
        default='./weights/yolov3.weights')

    parser.add_argument('-cfg', '--config',
        default='./cfg/yolov3.cfg')

    parser.add_argument('-l', '--labels',
        default='./data/coco.names')

    parser.add_argument('-c', '--confidence',
        default=0.5)

    parser.add_argument('-th', '--threshold',
        default=0.3)

    parser.add_argument('--download-model',
        default=False)

    parser.add_argument('-t', '--show-time',
        default=False)

    FLAGS, unparsed = parser.parse_known_args()


# Get the labels
    labels = open(FLAGS.labels).read().strip().split('\n')

# Intializing colors to represent each label uniquely
    colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# Load the weights and configutation to form the pretrained YOLOv3 model
    net = cv.dnn.readNetFromDarknet(FLAGS.config, FLAGS.weights)

    # Get the output layer names of the model
    layer_names = net.getLayerNames()
    layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [ ]:
# Infer real-time on webcam
vid = cv.VideoCapture(0)
count=0
while True:
            _, frame = vid.read()
            
            height, width = frame.shape[:2]

            if count == 0:
                frame, boxes, confidences, classids, idxs = infer_image(net, layer_names, \
                                    height, width, frame, colors, labels, FLAGS)
                count += 1
            else:
                frame, boxes, confidences, classids, idxs = infer_image(net, layer_names, \
                                    height, width, frame, colors, labels, FLAGS, boxes, confidences, classids, idxs, infer=False)
                count = (count + 1) % 6

            cv.imshow('webcam', frame)

            if cv.waitKey(1) & 0xFF == ord('q'):
                break
vid.release()
cv.destroyAllWindows()